In [1]:
import sqlite3
import pandas as pd

In [2]:
# Create Connection
con = sqlite3.connect('./data/Chinook_Sqlite.sqlite')

In [3]:
# Create Cursor
cursor = con.cursor()

In [4]:
# Create Query
album_count = """
SELECT COUNT()
FROM Album
"""

In [5]:
# Execute Query
cursor.execute(album_count)

In [6]:
# Return Query
count = cursor.fetchall()

In [7]:
print(type(count))
count

<class 'list'>


[(347,)]

In [8]:
# Quick helper function
def return_query(query, cursor):
    cursor.execute(query)
    return cursor.fetchall()

In [9]:
album_query = """
SELECT COUNT(*)
FROM Album
"""

In [10]:
al_list = return_query(album_query, cursor)
al_list

[(347,)]

# A reminder of how to read into pandas

In [11]:
albums_df = pd.read_sql(album_query, con)

albums_df

,COUNT(*)
0,347


In [12]:
al_list[0]

(347,)

In [13]:
al_list[0][0]

347

In [14]:
# Schema!
schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master

""", con)

schema_df

,type,name,tbl_name,rootpage,sql
0,table,Album,Album,19,CREATE TABLE [Album]\n(\n [AlbumId] INTEGER...
1,table,Artist,Artist,281,CREATE TABLE [Artist]\n(\n [ArtistId] INTEG...
2,table,Customer,Customer,386,CREATE TABLE [Customer]\n(\n [CustomerId] I...
3,table,Employee,Employee,392,CREATE TABLE [Employee]\n(\n [EmployeeId] I...
4,table,Genre,Genre,395,CREATE TABLE [Genre]\n(\n [GenreId] INTEGER...
5,table,Invoice,Invoice,396,CREATE TABLE [Invoice]\n(\n [InvoiceId] INT...
6,table,InvoiceLine,InvoiceLine,399,CREATE TABLE [InvoiceLine]\n(\n [InvoiceLin...
7,table,MediaType,MediaType,402,CREATE TABLE [MediaType]\n(\n [MediaTypeId]...
8,table,Playlist,Playlist,404,CREATE TABLE [Playlist]\n(\n [PlaylistId] I...
9,table,PlaylistTrack,PlaylistTrack,405,CREATE TABLE [PlaylistTrack]\n(\n [Playlist...


In [16]:
name_list = list(schema_df[ schema_df['type'] == 'table']['name'].values)
name_list

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

In [17]:
# This will create a new dataframe for each table in the database. This allows for easy exploration of the data.
# First, create the query with the format statement
# Create a temporary table with the pd.read_sql
# Create the variable name by concatenating the table name and '_df'
# Use the globals() to access the table_df name and assign the the temp df to it

for name in name_list:
    
    query = 'SELECT * from {}'.format(name)
    table_df = pd.read_sql(query, con)
    df_name=name+'_df'
    globals()[df_name]=table_df

In [18]:
# Create a list of dataframe names
df_list = [name+'_df' for name in name_list]

In [19]:
# Print out all the tables to provide a reference
# Please note that the exec() command is a rather powerful tool, and should be used lightly

for df in df_list:
    command = 'display({}.head())'.format(df)
    print(f'Table {df}')
    exec(command)

Table Album_df


,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


Table Artist_df


,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


Table Customer_df


...

,PlaylistId,Name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music


Table PlaylistTrack_df


,PlaylistId,TrackId
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392


Table Track_df


,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [ ]:
# Which Customer_id has the largest number of invoices?
query = """
None
"""
max_customer = pd.read_sql(query, con)
max_customer

In [ ]:
# What Song is most often purchased

query  = """
None
"""


trackcount = pd.read_sql(query, con)
trackcount


In [ ]:
# What is the average price of a track?

query = """
None
"""

avg_tracks = pd.read_sql(query, con)
avg_tracks


In [ ]:
# Which employee has the most direct reports?

query = """
None
"""

most_reports = pd.read_sql(query, con)
most_reports

In [ ]:
# In what city does the customer with the largest purchase amount live?

query = """
None
"""

largest = pd.read_sql(query, con)
largest

In [ ]:
# What country has the largest amount of customers?

query = pd.read_sql("""
None
""", con)

country_largest_customers = pd.read_sql(query, con)
country_largest_customers

In [ ]:
# What is the most popular song by country? Who is the artist?
query = """
None
"""

most_pop = pd.read_sql(query, con)
most_pop

In [ ]:
# What is the most common artist by playlist? 

# What is the name of their most popular song?

query = """
None
"""

common_artist = pd.read_sql(query, con)
common_artist

In [ ]:
# What song is in the most playlists?

# Who is the artist?

query = """
None
"""

most_playlists = pd.read_sql(query, con)
most_playlists